# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-2.28,71,6,6.78,SJ,1700947626
1,1,yellowknife,62.4560,-114.3525,-13.71,66,75,3.60,CA,1700947634
2,2,nova sintra,14.8667,-24.7167,22.44,73,28,8.62,CV,1700947634
3,3,puerto natales,-51.7236,-72.4875,9.25,66,36,9.06,CL,1700947635
4,4,reyes,-14.3167,-67.3833,26.26,76,100,0.38,BO,1700947635


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
Optimized_df = city_data_df.loc[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<27) & 
                                 (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
Optimized_df = Optimized_df.dropna()

# Display sample data
Optimized_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,lihue,21.9789,-159.3672,26.77,76,0,1.54,US,1700947660
56,56,wailua homesteads,22.0669,-159.3780,26.44,81,0,1.54,US,1700947662
66,66,jiwani,25.0500,61.7417,23.69,65,0,3.57,PK,1700947666
87,87,atar,20.5169,-13.0499,21.85,17,0,3.72,MR,1700947677
113,113,buckeye,33.3703,-112.5838,21.79,17,0,4.12,US,1700947639
204,204,tidjikja,18.5564,-11.4271,22.29,16,0,3.56,MR,1700947739
244,244,saint-pierre,-21.3393,55.4781,22.82,68,0,1.54,RE,1700947760
289,289,bansang,13.4333,-14.6500,26.34,18,0,2.00,GM,1700947785
380,380,ulladulla,-35.3500,150.4667,22.09,46,0,3.60,AU,1700947852
408,408,maceio,-9.6658,-35.7353,25.69,78,0,3.60,BR,1700947726


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
criteria = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = Optimized_df[criteria].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,lihue,US,21.9789,-159.3672,76,
56,wailua homesteads,US,22.0669,-159.3780,81,
66,jiwani,PK,25.0500,61.7417,65,
87,atar,MR,20.5169,-13.0499,17,
113,buckeye,US,33.3703,-112.5838,17,
204,tidjikja,MR,18.5564,-11.4271,16,
244,saint-pierre,RE,-21.3393,55.4781,68,
289,bansang,GM,13.4333,-14.6500,18,
380,ulladulla,AU,-35.3500,150.4667,46,
408,maceio,BR,-9.6658,-35.7353,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lihue - nearest hotel: Kauai Palms
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
jiwani - nearest hotel: No hotel found
atar - nearest hotel: فندق سكليل
buckeye - nearest hotel: Days Inn
tidjikja - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
bansang - nearest hotel: No hotel found
ulladulla - nearest hotel: Hotel Marlin
maceio - nearest hotel: Maceió Hostel e Pousada
tionk essil - nearest hotel: Campement Edonia
kiffa - nearest hotel: فندق كيفة


,City,Country,Lat,Lng,Humidity,Hotel Name
53,lihue,US,21.9789,-159.3672,76,Kauai Palms
56,wailua homesteads,US,22.0669,-159.3780,81,Hilton Garden Inn Kauai Wailua Bay
66,jiwani,PK,25.0500,61.7417,65,No hotel found
87,atar,MR,20.5169,-13.0499,17,فندق سكليل
113,buckeye,US,33.3703,-112.5838,17,Days Inn
204,tidjikja,MR,18.5564,-11.4271,16,No hotel found
244,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
289,bansang,GM,13.4333,-14.6500,18,No hotel found
380,ulladulla,AU,-35.3500,150.4667,46,Hotel Marlin
408,maceio,BR,-9.6658,-35.7353,78,Maceió Hostel e Pousada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)